In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
torch.cuda.is_available()

In [ ]:
np.random.seed(42)

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=10.0,max_zoom=1.5,max_lighting=0.5,max_warp=0.5,p_affine=0.75,p_lighting=0.75)


In [ ]:
mixup=0.2

In [ ]:
# loss_func = LabelSmoothingCrossEntropy()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()
loss_func = FocalLoss(gamma=2.5)

In [ ]:
alpha=0.99
mom=0.9
eps=1e-6

In [ ]:
opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)

In [ ]:
src = (ImageList.from_folder(path='train').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
data = src.transform(tfms,size=(640)).databunch(bs=24).normalize()

In [ ]:
learn = cnn_learner(data, models.densenet121,pretrained=False, metrics=[error_rate, accuracy, FBeta(beta=1,average='macro')],loss_func = loss_func,opt_func=opt_func,wd=1e-2)

In [ ]:
learn= learn.mixup(alpha=mixup)

In [ ]:
learn.bn_wd=False

In [ ]:
learn.true_wd=True

In [ ]:
learn.load('exp-12-round-1')

In [ ]:
data_704 = src.transform(tfms,size=(704)).databunch(bs=18).normalize()

In [ ]:
data_704.show_batch(rows=3, figsize=(12,12))

In [ ]:
learn.data= data_704

In [ ]:
learn = learn.to_fp16(dynamic=True)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=7.5e-7

In [ ]:
learn.fit_one_cycle(10, lr, div_factor=10, pct_start=0.3)

In [ ]:
learn.save('exp-13-round-1')